In [180]:
import pandas as pd
import numpy as np

# Procesamiento columna 'Descripción'

In [181]:
excel_file = pd.ExcelFile('../data/raw/data.xlsx')
sheets = excel_file.sheet_names
sheets

['Lista de equipos',
 'Historico calif 2_1(10 años)',
 'T-PAP(C)_T-ACE',
 'ARC(I)_ARC(D)',
 'DP',
 'OIL.TAP',
 'DIEL.PF_ DIEL',
 'OIL.CORR',
 'PRO-M',
 'M.OLTC']

In [182]:
import unidecode

def norm_descrpcion(s):
    s = s.lower()
    s = unidecode.unidecode(s)
    return s

converter_descripcion = {}
converter_descripcion['Descripción'] = norm_descrpcion

df_collection = {}
for name in sheets:
    df_collection[name] = pd.read_excel('../data/raw/data.xlsx', sheet_name=name, converters=converter_descripcion)

1. Separar cada fila en cada una de las descripciones. i.e : "realizar analisis dga, humedad y furanos" deben quedar dos filas exactamente iguales pero con cada uno de los registros. -> pd.explode(), explode a esa accion de expandir un dataset con base a una regla.

2. Aplicar un algoritmo de distancia de levehnstein. 

3. Análisis descriptivo que relacione cada equipo con las acciones. i.e: frecuencias de cada accion para cada equipo.

## Separación manual

In [183]:
def separacion_manual(dframe, originales, reemplazos, adiciones):
    df = dframe.copy()
    for i in range(len(originales)):
        index = df['Descripción'] == originales[i]
        if sum(index) > 0:
            instancias = df.loc[index].copy()
            df.loc[index,'Descripción'] = reemplazos[i]
            df = df.append(instancias, ignore_index = True)
            df.loc[len(df) - sum(index):,'Descripción'] = adiciones[i]
    return df




df_collection_separados = {}

excel_file_reemplazos = pd.ExcelFile('../data/reemplazos_manual/reemplazos_manual.xlsx')
sheets_reemplazos = excel_file_reemplazos.sheet_names
df_collection_manuales = {}

converter_manuales = {}
converter_manuales['Originales'] = norm_descrpcion
converter_manuales['Reemplazos'] = norm_descrpcion
converter_manuales['Adiciones'] = norm_descrpcion

for name in sheets_reemplazos:#sheets_reemplazos:
    df_collection_manuales[name] = pd.read_excel('../data/reemplazos_manual/reemplazos_manual.xlsx', sheet_name = name, converters = converter_manuales)
    index_manuales = ~df_collection_manuales[name].loc[:,'Reemplazos'].isna()
    originales = df_collection_manuales[name].loc[index_manuales, 'Originales'].values
    reemplazos = df_collection_manuales[name].loc[index_manuales, 'Reemplazos'].values
    adiciones = df_collection_manuales[name].loc[index_manuales, 'Adiciones'].values
    df_collection_separados[name] = separacion_manual(df_collection[name], originales, reemplazos, adiciones)

## Normalización por palabras claves

In [184]:
palabras_clave = {}
palabras_clave['revisar/cambiar motoventilador'] = ['motoventilador','ventilador','motoventiladores','ventiladores']
palabras_clave['realizar analisis dga y humedad seguimiento'] = ['dga']
palabras_clave['hacer seguimiento generacion anormal de gases'] = ['generacion','energizacion','gases']
palabras_clave['cambiar silica oltc'] = ['silica']
palabras_clave['cambiar filtro oltc'] = ['filtro']
palabras_clave['realizar secado oltc'] = ['secado']
palabras_clave['realizar muestreo-analisis fsco-qco oltc'] = ['aceite','muestra','fsco','qco']
palabras_clave['realizar regeneramiento aceite aislante'] = ['regeneramiento','regeneracion']
palabras_clave['adicionar irgamet39 a 100ppm'] = ['irgamet','pasivador','pasivar']
palabras_clave['temperatura devenado'] = ['calibrar indicador temperatura devanados']
palabras_clave['efectuar prueba de descargas parciales'] = ['descargas parc']
palabras_clave['seguimiento durante montaje'] = ['durante montaje']
palabras_clave['verificar condicion antes energizar oltc'] = ['energizar']
palabras_clave['realizar prueba estimacion humedad whrt'] = ['estimacion humedad']
palabras_clave['cambiar valvula alivio presion'] = ['alivio presion', 'alivio de presion','valvula alivio','valvula de alivio','valvula de sobre presion','valvula sobre presion','valvula de sobrepresion','valvula sobrepresion']
palabras_clave['presion subita'] = ['cambiar rele presion subita']
palabras_clave['revisar/cambiar contador cambiador tapsbiador'] = ['contador']
unicos_pc = {}

for name in sheets_reemplazos:
    unicos = pd.unique(df_collection_separados[name]['Descripción'])
    for i in range(len(unicos)):
        for key,value in palabras_clave.items():
            for word in value:
                if word in unicos[i]:
                    unicos[i] = key
                    break
    unicos_pc[name] = unicos

## Aplicando el algoritmo de distancia de Levenshtein

### Algoritmo de Levenshtein

In [185]:
def dist_lev(s1,s2):
    dist_matrix = np.zeros((len(s1) + 1, len(s2) + 1))
    dist_matrix[1:,0] = range(1,len(s1) + 1)
    dist_matrix[0,1:] = range(1,len(s2) + 1)
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i - 1] == s2[j - 1]:
                dist_matrix[i,j] = min(dist_matrix[i - 1, j] + 1, 
                                    dist_matrix[i - 1, j - 1], 
                                    dist_matrix[i, j - 1] + 1)
            else:
                dist_matrix[i,j] = min(dist_matrix[i - 1, j] + 1, 
                                    dist_matrix[i - 1, j - 1] + 1, 
                                    dist_matrix[i, j - 1] + 1)
    return dist_matrix[-1,-1]

### Aplicando y ordenando según distancia

In [179]:
# Extrayendo valores únicos:
ordenados_collection = {}
dist_ordenados_collection = {}
index_ordenados_collection = {}
for name in sheets_reemplazos:
    unicos = pd.unique(unicos_pc[name])
    #Construyendo matriz de distancia de Levenshtein
    dist_unicos = np.zeros((len(unicos), len(unicos)))
    for i in range(len(unicos)):
        for j in range(i + 1, len(unicos)):
            dist_unicos[i, j] = dist_lev(unicos[i], unicos[j])
            dist_unicos[j,i] = dist_unicos[i,j] 
    #Inizializar varibales
    # dist_unicos = np.array([[0,8,8,10.5],[1,0,4,3],[2,5,0,6],[8,10,7,0]])
    # unicos = [1,2,3,4]
    ordenados = ['']*len(unicos)
    dist_ordenados = np.zeros(len(unicos))
    index_ordenados = np.zeros(len(unicos))
    #Ubicando primer elemento
    max_dist_index = np.argmax(dist_unicos)
    max_dist_index = np.unravel_index(max_dist_index, np.shape(dist_unicos))
    row = max_dist_index[0]
    ordenados[0] = unicos[row]
    index_ordenados[0] = row
    dist_ordenados[0] = 0
    dist_unicos[row,:] = np.inf
    #Diagonal igual a infinito
    for i in range(len(unicos)):
        dist_unicos[i,i] = np.inf
    #Ubicando el resto de los elementos
    for i in range(len(unicos) - 1):
        min_dist = np.min(dist_unicos[:,row])
        dist_ordenados[i + 1] = dist_ordenados[i] + min_dist
        row = np.argmin(dist_unicos[:,row])
        ordenados[i + 1] = unicos[row]
        index_ordenados[i + 1] = row
        dist_unicos[row,:] = np.inf
    #Ubicándolos por hoja
    ordenados_collection[name] = ordenados
    dist_ordenados_collection[name] = dist_ordenados
    index_ordenados_collection[name] = index_ordenados


[[inf inf inf ... inf inf inf]
 [31. inf 32. ... 26. 16. 23.]
 [31. 32. inf ... 32. 36. 17.]
 ...
 [32. 26. 32. ... inf 24. 22.]
 [35. 16. 36. ... 24. inf 24.]
 [24. 23. 17. ... 22. 24. inf]]
[[inf inf inf inf inf inf]
 [31. inf 38. 32. 33. 17.]
 [39. 38. inf 34. 35. 37.]
 [33. 32. 34. inf 30. 31.]
 [36. 33. 35. 30. inf 27.]
 [24. 17. 37. 31. 27. inf]]
[[inf 27. 32. 34. 29. 10. 32. 23.]
 [inf inf inf inf inf inf inf inf]
 [32. 38. inf 31. 32. 32. 36. 21.]
 [34. 39. 31. inf 33. 34. 34. 24.]
 [29. 34. 32. 33. inf 30. 31. 28.]
 [10. 29. 32. 34. 30. inf 32. 25.]
 [32. 34. 36. 34. 31. 32. inf 31.]
 [23. 35. 21. 24. 28. 25. 31. inf]]
[[inf 10. 27. ... 24. 24. 13.]
 [10. inf 33. ... 27. 30. 19.]
 [27. 33. inf ... 23. 25. 31.]
 ...
 [24. 27. 23. ... inf 22. 27.]
 [24. 30. 25. ... 22. inf 31.]
 [13. 19. 31. ... 27. 31. inf]]
[[inf 25. 31. 28. 38. 27.]
 [25. inf 25. 27. 33. 25.]
 [inf inf inf inf inf inf]
 [28. 27. 32. inf 35. 28.]
 [38. 33. 39. 35. inf 32.]
 [27. 25. 28. 28. 32. inf]]
[[inf 32.

### Escribiendo los resultados en Excel

In [186]:
import xlsxwriter
workbook = xlsxwriter.Workbook('../data/reemplazos_manual/unicos_ordenados.xlsx')
for name in sheets_reemplazos:
    worksheet = workbook.add_worksheet(name)
    for i in range(len(ordenados_collection[name])): 
        worksheet.write(i,0,ordenados_collection[name][i])
        worksheet.write(i,1,ordenados_collection[name][i])
        worksheet.write(i,2,dist_ordenados_collection[name][i])
        worksheet.write(i,3,index_ordenados_collection[name][i])

workbook.close()

In [187]:
ordenados_collection['T-PAP(C)_T-ACE']

['realizar analisis dga y humedad seguimiento',
 'realizar analsisis de metanol en doble',
 'realizar analisis fiso-qc',
 'realizar muestreo-analisis fsco-qco oltc',
 'realizar muestreo y analisis furanos',
 'muestreo y analisis de furanos',
 'temperatura devenado',
 'revisar/cambiar motoventilador',
 'montaje de moto ventiladir',
 'revisar circuito control oltc',
 'revisar ventilacion forzada',
 'revisar control refrigeracion atr-1',
 'reemplazar controlador refrigeracion',
 'cambiar controlador temperatura',
 'calibrar el indicador de temperatura',
 'calibrar  indicador temperatura devanado',
 'revisar indicador temperatura fase c',
 'revision indicadores temperatura/t pozos',
 'cambiar indicador de nivel cuba ppal',
 'cambiar indicado de nivel cuba',
 'realizar cambio de indicador  nivel cuba',
 'hacer cambio de indicador nivel oltc fb',
 'repar sistema de refrigeracion  fase c',
 'revisar sistema de enfriamiento forzado',
 'revisar ruido anormal motovent. #40',
 'revisar ruido anor

In [188]:
dist_unicos[row,:] = np.inf